In [78]:
import sys
import os
# sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')

import numpy as np 
# from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn


from fasttext import FastVector


epochs = 50
trainable=True

In [2]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [3]:
# changing all deriv and part to misc. with BIO
def modify_german_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset


def modify_english_labels(dataset):
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            label =  label[:-1]
            word[1]= label
    return dataset
                

In [4]:

# preproecessing data from GermEval
def get_german_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences




In [31]:
def normalized(a, axis=-1, order=2):
    """Utility function to normalize the rows of a numpy array."""
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

def make_training_matrices(source_dictionary, target_dictionary, bilingual_dictionary):
    """
    Source and target dictionaries are the FastVector objects of
    source/target languages. bilingual_dictionary is a list of 
    translation pair tuples [(source_word, target_word), ...].
    """
    source_matrix = []
    target_matrix = []

    for (source, target) in bilingual_dictionary:
        if source in source_dictionary and target in target_dictionary:
            source_matrix.append(source_dictionary[source])
            target_matrix.append(target_dictionary[target])

    # return training matrices
    return np.array(source_matrix), np.array(target_matrix)

def learn_transformation(source_matrix, target_matrix, normalize_vectors=True):
    """
    Source and target matrices are numpy arrays, shape
    (dictionary_length, embedding_dimension). These contain paired
    word vectors from the bilingual dictionary.
    """
    # optionally normalize the training vectors
    if normalize_vectors:
        source_matrix = normalized(source_matrix)
        target_matrix = normalized(target_matrix)

    # perform the SVD
    product = np.matmul(source_matrix.transpose(), target_matrix)
    U, s, V = np.linalg.svd(product)

    # return orthogonal transformation which aligns source language to the target
    return np.matmul(U, V)

In [5]:

english_trainSentences = readfile("data/train.txt")
english_devSentences = readfile("data/valid.txt")
english_testSentences = readfile("data/test.txt")

In [6]:
print(english_trainSentences[0])

[['EU', 'B-ORG\n'], ['rejects', 'O\n'], ['German', 'B-MISC\n'], ['call', 'O\n'], ['to', 'O\n'], ['boycott', 'O\n'], ['British', 'B-MISC\n'], ['lamb', 'O\n'], ['.', 'O\n']]


In [7]:
english_trainSentences.extend(english_devSentences)
english_trainSentences.extend(english_testSentences)
trainSentences = english_trainSentences

In [8]:
print(trainSentences[0])

[['EU', 'B-ORG\n'], ['rejects', 'O\n'], ['German', 'B-MISC\n'], ['call', 'O\n'], ['to', 'O\n'], ['boycott', 'O\n'], ['British', 'B-MISC\n'], ['lamb', 'O\n'], ['.', 'O\n']]


In [9]:
trainSentences = modify_english_labels(trainSentences)
# trainSentences = addCharInformatioin(trainSentences)

In [10]:
print(trainSentences[0])

[['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O']]


In [11]:
trainSentences = addCharInformatioin(trainSentences)


In [12]:
print(trainSentences[-2])

[['He', ['H', 'e'], 'O'], ['guided', ['g', 'u', 'i', 'd', 'e', 'd'], 'O'], ['Ireland', ['I', 'r', 'e', 'l', 'a', 'n', 'd'], 'B-LOC'], ['to', ['t', 'o'], 'O'], ['two', ['t', 'w', 'o'], 'O'], ['successive', ['s', 'u', 'c', 'c', 'e', 's', 's', 'i', 'v', 'e'], 'O'], ['World', ['W', 'o', 'r', 'l', 'd'], 'B-MISC'], ['Cup', ['C', 'u', 'p'], 'I-MISC'], ['finals', ['f', 'i', 'n', 'a', 'l', 's'], 'O'], ['tournaments', ['t', 'o', 'u', 'r', 'n', 'a', 'm', 'e', 'n', 't', 's'], 'O'], ['and', ['a', 'n', 'd'], 'O'], ['to', ['t', 'o'], 'O'], ['the', ['t', 'h', 'e'], 'O'], ['1988', ['1', '9', '8', '8'], 'O'], ['European', ['E', 'u', 'r', 'o', 'p', 'e', 'a', 'n'], 'B-MISC'], ['championship', ['c', 'h', 'a', 'm', 'p', 'i', 'o', 'n', 's', 'h', 'i', 'p'], 'O'], ['finals', ['f', 'i', 'n', 'a', 'l', 's'], 'O'], ['in', ['i', 'n'], 'O'], ['Germany', ['G', 'e', 'r', 'm', 'a', 'n', 'y'], 'B-LOC'], [',', [','], 'O'], ['after', ['a', 'f', 't', 'e', 'r'], 'O'], ['the', ['t', 'h', 'e'], 'O'], ['Irish', ['I', 'r', 'i'

In [14]:
testSentences = get_german_sentences('/srv/Resources/GermEVAL/NER-de-test.tsv')
testSentences = modify_german_labels(testSentences)


In [15]:
print(testSentences[0])

[['1951', 'O'], ['bis', 'O'], ['1953', 'O'], ['wurde', 'O'], ['der', 'O'], ['nördliche', 'O'], ['Teil', 'O'], ['als', 'O'], ['Jugendburg', 'O'], ['des', 'O'], ['Kolpingwerkes', 'B-MISC'], ['gebaut', 'O'], ['.', 'O']]


In [16]:
testSentences = addCharInformatioin(testSentences)

In [17]:
print(testSentences[0])

[['1951', ['1', '9', '5', '1'], 'O'], ['bis', ['b', 'i', 's'], 'O'], ['1953', ['1', '9', '5', '3'], 'O'], ['wurde', ['w', 'u', 'r', 'd', 'e'], 'O'], ['der', ['d', 'e', 'r'], 'O'], ['nördliche', ['n', 'ö', 'r', 'd', 'l', 'i', 'c', 'h', 'e'], 'O'], ['Teil', ['T', 'e', 'i', 'l'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['Jugendburg', ['J', 'u', 'g', 'e', 'n', 'd', 'b', 'u', 'r', 'g'], 'O'], ['des', ['d', 'e', 's'], 'O'], ['Kolpingwerkes', ['K', 'o', 'l', 'p', 'i', 'n', 'g', 'w', 'e', 'r', 'k', 'e', 's'], 'B-MISC'], ['gebaut', ['g', 'e', 'b', 'a', 'u', 't'], 'O'], ['.', ['.'], 'O']]


In [18]:
devSentences = get_german_sentences('/srv/Resources/GermEVAL/NER-de-dev.tsv')
devSentences = modify_german_labels(devSentences)
devSentences = addCharInformatioin(devSentences)

In [19]:
print(devSentences[1])

[['1980', ['1', '9', '8', '0'], 'O'], ['kam', ['k', 'a', 'm'], 'O'], ['der', ['d', 'e', 'r'], 'O'], ['Crown', ['C', 'r', 'o', 'w', 'n'], 'B-MISC'], ['als', ['a', 'l', 's'], 'O'], ['Versuch', ['V', 'e', 'r', 's', 'u', 'c', 'h'], 'O'], ['von', ['v', 'o', 'n'], 'O'], ['Toyota', ['T', 'o', 'y', 'o', 't', 'a'], 'B-ORG'], [',', [','], 'O'], ['sich', ['s', 'i', 'c', 'h'], 'O'], ['in', ['i', 'n'], 'O'], ['der', ['d', 'e', 'r'], 'O'], ['Oberen', ['O', 'b', 'e', 'r', 'e', 'n'], 'O'], ['Mittelklasse', ['M', 'i', 't', 't', 'e', 'l', 'k', 'l', 'a', 's', 's', 'e'], 'O'], ['zu', ['z', 'u'], 'O'], ['etablieren', ['e', 't', 'a', 'b', 'l', 'i', 'e', 'r', 'e', 'n'], 'O'], [',', [','], 'O'], ['auch', ['a', 'u', 'c', 'h'], 'O'], ['nach', ['n', 'a', 'c', 'h'], 'O'], ['Deutschland', ['D', 'e', 'u', 't', 's', 'c', 'h', 'l', 'a', 'n', 'd'], 'B-LOC'], ['.', ['.'], 'O']]


In [20]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
#             print(label)
            try:
                labelSet.add(label)
            except Exception as e:
                print(label)
                continue
            words[token] = True
            

In [21]:
print(labelSet)

{'B-ORG', 'I-MISC', 'I-PER', 'B-PER', 'B-MISC', 'B-LOC', 'I-ORG', 'O', 'I-LOC'}


In [22]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [23]:
print(label2Idx)

{'B-ORG': 0, 'B-PER': 3, 'I-MISC': 1, 'I-ORG': 6, 'O': 7, 'I-LOC': 8, 'I-PER': 2, 'B-MISC': 4, 'B-LOC': 5}


In [24]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [25]:
german_words = {}

for dataset in [testSentences, devSentences]:
    for sentence in dataset:
        for token, char, label in sentence:
            german_words[token]=True
            
            

In [28]:
string_german_words = ' '.join(german_words.keys())
print(string_german_words)

unit( Senden Anschließend Mode symbolischen Präsidentschaftskandidat Administration spektakulären Katholik Luftballons Deportationszug Regelmäßigkeit abberufen Spielen Birnbach 13 Schweigegelübtes Eiszapfen Lichtenrade Facettenaugen 125-jährige Annahme Vandalismus Verschiedene Helfer überzeugend Gotteshauses bin Energiewende Behaarung Flucht Selbst Bundestrainer Schwangere 10 gelb Spitznamen trockenen Hülkenberg Steffen Leergut Steine Queen Cézanne Fahrenden Informatiker schreibt aufgesetzten superschnelle Patterson Adlige Andrang Dam Kádárs Barredo Bauarbeiten 8,75 23,2 Banks Konfetti-Schnipsel Mobil- 29 32 romanische Vor-Ort-Verfügbarkeit Knochen­bruchstücken Konjunkturprogramm Antisemit Wirtschaftswoche günstige Gedanke kletterte Abzugsverbot Stiller beschloss Ministern Relapse Hedge-Fonds Somit Mikulic schwächer Akteure Hotbeds Parkstrategie Namen Mineirão einigermassen Film-Dokumentation finde Auszeichnung Dehner Szenen Schmolllippen Vilanek <VOW.ETR> 34-jährige Plenum Standesämte

In [29]:
english_words = {}
for dataset in [trainSentences]:
    for sentence in dataset:
        for word, char, label in sentence:
            english_words[word] = True
    

In [30]:
string_english_words = ' '.join(english_words.keys())
print(string_english_words)

IL brutal Zelenskaja founder 13 reconciling presiding Bergamo-based 229 12-footer abetting S.A. wedge practices 199 1,820,000 8167 vintage Vladikavkaz Gift Brassard 63.67 Anthuan SunGard halves protesters skilful colleague Housecall ads kilo-bars 1mo received Gagiano 10 Aitken occasions Steffen Hebrew Jonty 2:02.40 unlikely Pieters 116 Andean administrator justice lashed dense momentous Hondo 721 29 32 UWP-Hachette Dohuk guides tiebreak a.m. 225-share chaired Eduard DAVEY Sugiyama easing Zellner Ghent Savicevic Opava Grunde McGregor pending police 4.252 existed prey 1.38 militia precious Cunha cold relinquish 10.03 Giffin contracting cabaret J.S. Noel Middle expects uphill Klara 11-9 Tulio AWARD Tunisia filled briefly 6-260 27th distinguished protests Ponce 4,900 1017 therapy teenage veiled Pot cries endeavours Jiri 9-3 1996-A villain Claudio uninspiring 18-18 Capt hills lending Guillaume internationally Faulding Richardson 2/94 3. 38:34.436 Gagne Bjorkman watchdog Tolley socks Campbel

In [63]:
os.getcwd()

'/srv/jindal/NER-Bi-LSTM-CNN'

In [80]:
em_dict = FastVector(vector_file='/srv/fastText/wiki.en.vec')
# ru_dictionary = FastVector(vector_file='/srv/fastText/wiki.de.vec')

reading word vectors from /srv/fastText/wiki.en.vec


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 6668: ordinal not in range(128)